In [1]:
import warnings;
warnings.filterwarnings('ignore');
import pandas as pd

# **Core promoter prediction from DNA sequence**

Promoter detection in the human genome focuses on identifying proximal promoter regions, crucial sequences responsible for initiating transcription. Accurate identification of these sites is essential for advancing our understanding of gene regulation mechanisms and identifying the genomic basis of numerous diseases. This assignment involves training and evaluating a core promoter prediction model, which aims to predict the central core promoter region closest to the transcription start site (TSS) and start codon.

Promoter sequences are extracted as -34 to +35 base pairs around the TSS from TATA promoters, sourced from the Eukaryotic Promoter Database (EPDnew) [1]. The non-promoter class is constructed using equal-sized randomly selected sequences outside promoter regions that contain the TATA motif (TATA non-promoters).

The goal of this assignment is to develop a model that accurately distinguishes core promoter regions from non-promoter regions, thereby contributing to our understanding of the fundamental elements governing gene expression.

First we load the train and test data:



In [2]:
data_train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/practicum_core_promoter_prediction/main/prom_core_tata/train.csv")
data_test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/practicum_core_promoter_prediction/main/prom_core_tata/test.csv")

data_train.head(5)

,sequence,label
0,ACTACAAATGGACGAGAGAGGCGGCCGTCCATTAGTTAGCGGCTCC...,1
1,AAAATATAGGCCGGGGTACCTCAGCCGGAAGGGACTTCAGTTAGTG...,1
2,CCTACATAAGTCCCTGTATAAAGTCACTGACCCATTTGCACTGCTG...,1
3,AGTTTAAAAGCCAGCCAGTCATACTAAAAAAAAGAATTCAGGTTTT...,0
4,GGGATAAGAAGGACAGAGAGAGACTGTAGGAAGTCAAGGGGTGGAG...,0


There are just two columns.

The column "sequence" contains the -34 to +35 base pairs around the candididate TSS.

The column "label" contains the label. There are two classes: 1 for a core promoter region and 0 for not core promoter region.

*Q1: How many sequences does the train set contain for each class?*

In [ ]:
###Start code here


###End code here

### **Feature engineering**

Feature engineering is a crucial process in machine learning that involves creating new features or modifying existing ones to improve the performance of a model. It bridges the gap between raw data and the algorithms used for predictive modeling.

To derive features from the sequence column, the train and test datasets are initially concatenated into a single Pandas DataFrame. This unified DataFrame enables the application of feature engineering processes uniformly. Subsequently, the original train and test DataFrames can be reconstructed from this consolidated DataFrame.

*Q2: Use the Pandas function `concat()` to concatenate the train and test data into a DataFrame called `data`. The train data should be the first rows, with the test data beneath those rows:*

In [ ]:
###Start code here

data =

###End code here

data

*Q3: Pop the `label` column from the `data` DataFrame and assigned it to variable `y`*:

In [ ]:
###Start code here

y =

###End code here

y

Now, it is necessary to represent the local context DNA sequence as a feature vector appropriate for model fitting.

Initially, a feature is created for each nucleotide position within the local context DNA sequence.

The [pandas.Series.str.split](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html) function splits a string in a column (pandas.Series) from the beginning, at the specified delimiter string:

In [ ]:
data_features = data['sequence'].str.split('', expand=True).iloc[:,1:-1]

data_features

Next, we should assign more meaningful names to the columns:

In [ ]:
data_features.columns = ["%i"%i for i in range(-34,0,1)] + ["%i"%i for i in range(0,36,1)]

data_features

The `sklearn.preprocessing.OrdinalEncoder` encode categorical features as an integers.

The folloing code creates a Pandas DataFrame `data_features_int_encoding` from `data_features` by applying the `OrdinalEncoder` to map each categorical feature value to an integer. Note the the `.transform()` function in scikit-learn returns a numpy datastructure, wo we need to convert the result back to a Pandas DataFrame.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()

enc.fit(data_features)

data_features_int_encoding = pd.DataFrame(enc.transform(data_features),columns=data_features.columns)

data_features_int_encoding

Finally, we reconstruct the train and test DataFrames based on the number of datapoints in the train data:

In [ ]:
data_features_int_encoding_train = data_features_int_encoding.iloc[:len(data_train),:]
data_features_int_encoding_test = data_features_int_encoding.iloc[len(data_train):,:]

y_train = y.iloc[:len(data_train)]
y_test = y.iloc[len(data_train):]

### **Model fitting**

Cross-validation (CV) is used when evaluating the performance of a machine learning model to ensure it generalizes well to unseen data, by systematically partitioning the data into training and validation sets multiple times. It is particularly useful in cases of limited data, where splitting into separate training and test sets might not be feasible or reliable.

*Q4: Estimate the generalization performance of a logisitc regression model with hyperparameter $C=0.1$ on `data_features_int_encoding_train` using 10-fold CV. Use `sklearn.model_selection.cross_val_score` to compute the accuracy for each test fold in the CV. Return the result in `scores`:*

In [ ]:
###Start code here

###End code here

print(scores.mean())
print(scores.std())


Hyperparameter tuning is the process of optimizing the parameters that govern the learning process of a machine learning model, which are not learned from the data but set prior to training. This process aims to enhance the model's performance by systematically searching through a predefined space of hyperparameters to find the most effective combination.

We set hyperparameter $C=1$ for the logistic regression model.

*Q5: Use `sklearn.model_selection.GridSearchCV` to find the best performing value for $C$ from the `search_space` search space. Create a GridSearchCV object called `grid_search` and optimize $C$:*

In [ ]:
from sklearn.model_selection import GridSearchCV

search_space =  {
                "C":[0.0001,0.001,0.01,0.1,1,10,100]
                }

###Start code here

###End code here

print(grid_search.best_estimator_)
print(grid_search.best_score_)

*Q6: Fit a logistic regression model with the obtained optimal value $C$ on the train set:*

In [ ]:
###Start code here

###End code here

*Q7: Compute the predictions of this model on the test set. Write these predictions to `predictions`:*

In [ ]:
###Start code here

###End code here

Scikit-learn offers many metrics to evaluate model predictions. These functions are implemented in `sklearn.metrics`.

*Q8: Compute the accuracy of the predictions in `predictions`:*

In [ ]:
from sklearn import metrics

###Start code here

score_acc =

###End code here

score_acc

### **Feature engineering: one-hot feature encoding**

One-hot encoding is a technique used to convert categorical variables into a binary matrix representation where each unique category is represented by a separate column with binary values (0 or 1). It is particularly useful for handling nominal categorical data in machine learning models, ensuring that the model interprets these categories correctly without implying any ordinal relationship.

*Q10: Use the Pandas function `get_dummies()` to compute one-hot encoded features. Make sure the feature values are `float`:*

In [ ]:
###Start code here

data_features_onehot_encoding =

###End code here

data_features_onehot_encoding

*Q11: Optimize $C$ of a logistic regression model from the train set part of `data_features_onehot_encoding`:*

In [ ]:
###Start code here

###End code here

*Q12: Fit a logistic regression model with the obtained optimal value $C$ on the train set:*

In [ ]:
###Start code here

###End code here

*Q13: What is the accuracy on the test data in `data_features_onehot_encoding`?*

In [ ]:
###Start code here

###Start code here

score_acc

### **Feature importance**

In scikit-learn, a fitted logistic regression model has the fitted modelparameter values stored in `.coef_[0]`:

In [ ]:
#THe following code assumes "model" is the fitted logistic regression model
print(model.coef_[0])

This is one modelparameter for each feature (plus the interecept, which is not in `.coef_[0]`).

Recall that in logistic regression a prediction is made by multiplying each fitted modelparameter with the corresponding feature value, summing them and then squeezing this sum between 0 and 1 with the sigmoid function.

Since all features have values 0 or 1, the modelparameter values indicate the contribution (importance) of a feature to the prediction.

The following code puts the feature names and modelparameter values in a new DataFrame:

In [ ]:
F_importances = []

for feature_name, modelparameter in zip(data_features_onehot_encoding.columns,model.coef_[0]):
    F_importances.append([feature_name,modelparameter])

F_importances = pd.DataFrame(F_importances,columns=["feature_name","importance"])
F_importances.head()

*Q14: What are the 20 features considered most important by the fitted logistic regression model?*

In [ ]:
#Start code here

#End code here

### **Feature selection**

`sklearn.feature_selection.SelectFromModel` is a feature selection method that uses a trained model to identify and retain only the most important features, based on the model's learned feature importances or coefficients. This technique helps in reducing the dimensionality of the dataset, improving model performance, and preventing overfitting by focusing on the most relevant features.

*Q15: Apply `sklearn.feature_selection.SelectFromModel` to return the 20 most important features from the previously fitted logistic regession model:*

In [ ]:
from sklearn.feature_selection import SelectFromModel

#Start code here


selected_feature_names =

#End code here

print("Selected features:", selected_feature_names)

*Q16: Are these the same features as were selected in Q15?*

*Answer:*

*Q17: Optimize and fit a logistic regression on the train set that uses the selected 20 most important features only. What is the accuracy of this model in the test set?*

In [ ]:
#Start code here

#End code here

Recursive Feature Elimination (RFE) is a feature selection method that recursively removes the least important features based on the model's coefficients or feature importances until the desired number of features is reached. This iterative process helps in identifying the most relevant features for improving model performance and reducing complexity.

*Q18: Use `sklearn.feature_selection.RFE` to select the 20 most important features (according to the RFE algorithm) from `data_features_onehot_encoding_train` for logistic regresion model with $C=0.001$. Set the `step` hyperparameter of RFE to 1. Are these same features selected as in Q15?*

In [ ]:
from sklearn.feature_selection import RFE

#Start code here

#End code here

*Answer:*

*[1] Dreos, R., Ambrosini, G., Perier, R. C., & Bucher, P. (2013). The Eukaryotic Promoter Database: expansion of EPDnew and new promoter analysis tools. Nucleic Acids Research, 41(D1), D50-D56.*